In [1]:
import requests
import pandas as pd

In [2]:
# system config
# source: dpmreporter.disaster.go.th/portal/
start_year = 2018
end_year = 2024

In [3]:
current_year = start_year

for i_year in range(end_year - start_year + 1):
    
    url = rf'https://dpmreporter.disaster.go.th/portal/services/disaster_statistics/get_stat_news.php?type=3&date_start={current_year}&date_end={current_year}'
    
    response = requests.post(url)
    response = response.json()

    data = response['data']['data']['data']

    if current_year == start_year:
        df = pd.json_normalize(data)
        df = df.assign(year = current_year)
    
    else:
        buffer_df = pd.json_normalize(data)
        buffer_df = buffer_df.assign(year = current_year)
        df = pd.concat([df, buffer_df], axis=0, ignore_index=True)

    current_year += 1

In [4]:
df = df.dropna().reset_index(drop=True)
df = df.drop(['id', 'color', 'percent'], axis=1)
df = df.rename(columns={'name': 'risk_type', 'news_province': 'pname', 'y': 'n_occurance', 'province_id': 'pid'})
df = df.loc[:, ['year', 'pid', 'pname', 'risk_type', 'n_occurance']]

df.pid = df.pid.astype(int)

In [5]:
p_loc_df = pd.read_csv(r'archive\dataset\ddpm\DDPM_arcrest_admin_province_csv.csv')
p_loc_df = p_loc_df.drop(['OBJECTID', 'PROV_NAM_T', 'PROV_NAM_E', 'ProvinceID', 'ProvinceTH', 'ProvinceEN', 'F4Region', 'MetroRegio', 'Units'], axis=1)
df = df.merge(p_loc_df, how='left', left_on='pid', right_on='PROV_CODE').drop(['PROV_CODE'], axis=1)
df = df.loc[:, ['year', 'pid', 'pname', 'FormalRegi', 'lat', 'lon', 'risk_type', 'n_occurance']]

In [39]:
df.to_csv('dataset/do_2018_2024.csv', encoding='utf-8', index=False)